# TF Core Tutorial

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.1.0'

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


A **placeholder** is a promise to provide a value later.

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

In [7]:
print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1,3], b:[2, 4]}))

7.5
[ 3.  7.]


In [8]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


**Variables** allow us to add trainable parameters to a graph.

In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [10]:
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


A **loss function** measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:

In [12]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like **tf.assign**. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:

In [13]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


# tf.train API

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [15]:
sess.run(init) # reset values to incorrect defaults.
print(sess.run([W,b]))

[array([ 0.30000001], dtype=float32), array([-0.30000001], dtype=float32)]


In [16]:
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

In [17]:
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# tf.contrib.learn

In [18]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

In [19]:
# Declare list of features. We only have one real-valued feature. 
# There are many other types of columns that are more complicated and useful.

In [20]:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
print(features)

[_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)]


In [21]:
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_model_dir': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_task_type': None, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000024CD0A2B70>, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 0, '_task_id': 0, '_tf_random_seed': None, '_master': '', '_is_chief': True}


In [22]:
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train,
                                              batch_size=4,
                                              num_epochs=1000)

eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)

In [23]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Program Files\Microsoft\R Server\R_SERVER\DeployR\rserve\workdir\tmp\tmp6nbfngmm\model.ckpt.
INFO:tensorflow:loss = 3.75, step = 1
INFO:tensorflow:global_step/sec: 910.791
INFO:tensorflow:loss = 0.053594, step = 101 (0.110 sec)
INFO:tensorflow:global_step/sec: 853.024
INFO:tensorflow:loss = 0.00533317, step = 201 (0.119 sec)
INFO:tensorflow:global_step/sec: 980.12
INFO:tensorflow:loss = 0.00145668, step = 301 (0.100 sec)
INFO:tensorflow:global_step/sec: 865.946
INFO:tensorflow:loss = 0.00013352, step = 401 (0.117 sec)
INFO:tensorflow:global_step/sec: 970.591
INFO:tensorfl

LinearRegressor(params={'joint_weights': False, 'optimizer': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x00000024CD0A2E80>, 'gradient_clip_norm': None, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)]})

In [24]:
# Here we evaluate how well our model did.
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-06-04:01:19
INFO:tensorflow:Restoring parameters from C:\Program Files\Microsoft\R Server\R_SERVER\DeployR\rserve\workdir\tmp\tmp6nbfngmm\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-07-06-04:01:20
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 2.94974e-08
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supp

### custom model

In [29]:
import numpy as np
import tensorflow as tf

In [30]:
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

In [31]:
estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_model_dir': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_task_type': None, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000024D1EEFDD8>, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 0, '_task_id': 0, '_tf_random_seed': None, '_master': '', '_is_chief': True}


In [32]:
# train
estimator.fit(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Program Files\Microsoft\R Server\R_SERVER\DeployR\rserve\workdir\tmp\tmp6bi9bfio\model.ckpt.
INFO:tensorflow:loss = 14.3702258726, step = 1
INFO:tensorflow:global_step/sec: 1089.63
INFO:tensorflow:loss = 0.0904014091324, step = 101 (0.092 sec)
INFO:tensorflow:global_step/sec: 1303.28
INFO:tensorflow:loss = 0.0149323236416, step = 201 (0.078 sec)
INFO:tensorflow:global_step/sec: 1388.87
INFO:tensorflow:loss = 0.000362417927639, step = 301 (0.073 sec)
INFO:tensorflow:global_step/sec: 1351.34
INFO:tensorflow:loss = 0.000127409928532, step = 401 (0.073 sec)
INFO:tensorflow:global_step/sec: 1388.93
INFO:tensorflow:loss = 7.04340498649e-06, step = 501 (0.073 sec)
INFO:tensorflow:global_step/sec: 1369.84
INFO:tensorflow:loss = 1.75584650496e-06, step = 601 (0.072 sec)
INFO:tensorflow:global_step/sec: 1388.89
INFO:tensorflow:loss = 6.70323333947e-08, step = 701 (0.073 sec)
INFO:tensorflow:global_step/s